## Key Topics Extraction and Contextual Sentiment of Hotel Reviews

## 1. Preprocess

In [26]:
import os
from os import listdir
import pandas as pd
#!pip install plotly
import plotly.graph_objs as go
df = pd.read_csv('/Users/lancy/Desktop/Nlp project/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv', header=0)
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'],
      dtype='object')

In [27]:
df.drop(columns=['dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'keys', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'], inplace = True)
df = df.rename(index=str, columns={'reviews.rating':'rating', 'reviews.sourceURLs': 'reviews_sourceURLs', 'reviews.text':'reviews_text', 'reviews.title': 'reviews_title'})

In [28]:
df.head()

id               city country  latitude  longitude  \
0  AWE2FvX5RxPSIh2RscTK             Goleta      US  34.44178 -119.81979   
1  AVwcj_OhkufWRAb5wi9T  Carmel by the Sea      US  36.55722 -121.92194   
2  AVwcj_OhkufWRAb5wi9T  Carmel by the Sea      US  36.55722 -121.92194   
3  AVwcj_OhkufWRAb5wi9T  Carmel by the Sea      US  36.55722 -121.92194   
4  AVwcj_OhkufWRAb5wi9T  Carmel by the Sea      US  36.55722 -121.92194   

                                     name postalCode province  rating  \
0       Best Western Plus South Coast Inn      93117       CA       3   
1  Best Western Carmel's Town House Lodge      93921       CA       4   
2  Best Western Carmel's Town House Lodge      93921       CA       3   
3  Best Western Carmel's Town House Lodge      93921       CA       5   
4  Best Western Carmel's Town House Lodge      93921       CA       2   

                                  reviews_sourceURLs  \
0  https://www.tripadvisor.com/Hotel_Review-g3243...   
1  https://www.tripadvisor.com/Hotel_Review-g3217...   
2  https://www.tripadvisor.com/Hotel_Review-g3217...   
3  https://www.tripadvisor.com/Hotel_Review-g3217...   
4  https://www.tripadvisor.com/Hotel_Review-g3217...   

                                        reviews_text  \
0  This hotel was nice and quiet. Did not know, t...   
1  We stayed in the king suite with the separatio...   
2  Parking was horrible, somebody ran into my ren...   
3  Not cheap but excellent location. Price is som...   
4  If you get the room that they advertised on th...   

                                       reviews_title  
0                            Best Western Plus Hotel  
1  Clean rooms at solid rates in the heart of Carmel  
2                                           Business  
3                                          Very good  
4                       Low chance to come back here

In [29]:
f= open("hotel_review.txt","w+")
for i in range(df.shape[0]):
    f.write(df.reviews_text[i])
    f.write("\n")
f.close()

In [30]:
with open("hotel_review.txt") as fp: 
    reviews = []
    Lines = fp.readlines() 
    for line in Lines[:5]: 
        reviews.append(line)

In [31]:
reviews

['This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.\n',
 "We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More\n",
 "Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.\n",
 'Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental bre

## 2. Amenity Extraction

In [32]:
import nltk.data
from os import listdir
from os.path import isfile, join
from nltk.util import bigrams 
from nltk.tokenize import TreebankWordTokenizer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()
import numpy as np
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
#stop_words = stopwords.words('english')

In [33]:
#check if the element passed in is a float
def isfloat(value):
  try:
    float(value)
    #return true if it can be cast to a float
    return True
  except ValueError:
    return False

In [34]:
stop_words = ['in','of','at','a','the']
def process_text(text):
    #tokenzie the text as sentences
    punkt_sentences = sentence_tokenizer.tokenize(text)
    #tokenize as lists of sentence lists of words
    sentences_words = [treebank_tokenizer.tokenize(sentence) for sentence in punkt_sentences]
    #get a list of words without punctuations
    tokens = [word for sentence in sentences_words for word in sentence if word == 'A/C' or word.isalpha()]
    #lemmatization 
    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    #remove the stopwords
    tokens = [word for word in tokens if not word in stop_words]
    #remove numbers
    tokens = [word for word in tokens if isfloat(word == False)]
    #generate bigrams
    bigrams = nltk.bigrams(tokens)
    return tokens,list(bigrams)

### Train a sentiment classifer

In [35]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
 
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)
 
#print(negcutoff)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
#print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))
 
classifier = NaiveBayesClassifier.train(trainfeats)
#print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
#classifier.show_most_informative_features()

In [36]:
import pickle
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [37]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [38]:
from nltk.corpus import sentiwordnet as swn
def senti(bigram):
    #TODO 'have'
    if bigram.lower() == 'have' or bigram.lower() == 'free':
        return True
    happy = swn.senti_synsets(bigram)
    ps = 0
    ns = 0
    for i in list(happy):
        ps += i.pos_score()
        ns += i.neg_score()
    return max(ps,ns) == ps


In [39]:
amenity_list = ['A/C','fan','parking','coffee','breakfast','pool','clean',
             'kitchen','internet','computer','recreation','exercise',
             'vending','locker','dryer','laundry']
def amenity_extraction_hotel(hotel):
    amenities = {}
    for text in df[df['name'] == hotel]['reviews_text']:
        tokens,bigrams = process_text(text)
        for word in tokens:
            for amenity in amenity_list:
                if word.lower() == amenity.lower():
                    if word not in amenities.keys():
                        amenities.update({word:0})
                    for bi in bigrams:
                        if bi[1].lower == word.lower:
                            #print(bi[0])
                            #print(senti(bi[0]))
                            if senti(bi[0]):
                                #print(bi[0])
                                amenities[word] += 1
                            else:
                                print(bi[0])
                                amenities[word] -= 1               
    return amenities

In [40]:
amenity_extraction_hotel(df['name'][4])

no


{'clean': 2,
 'Parking': 0,
 'breakfast': 2,
 'coffee': 1,
 'vending': 1,
 'A/C': -1,
 'fan': 1,
 'pool': 1}

### Extract amenties for all the hotels

In [41]:
hotel_amenities = {}
for hotel in df['name'].unique():
    amenities = amenity_extraction_hotel(hotel)
    hotel_amenities.update({hotel:amenities})

no
swimming
job
cold
A
Broken
trip
always
small
Comfy
unit
not
job
spite
front
normal
PLENTY
did
no
average
night
plenty
plenty
quiet
bus
by
blow
swimming
had
sleeping
handicap
Bar
stayed
visit
by
tropical
somewhere
no
handicap
always
No
slow
no
by
leave
off
need
do
an
plenty
face
not
swimming
by
swimming
waffle
swimming
leave
leave
an
fresh
too
late
had
unaware
by
inconvenience
overlooking
next
quiet
leave
night
managed
no
outrageous
couple
small
experience
experience
pain
away
no
cheap
had
dryer
limited
Plenty
dirty
no
retreat
by
swimming
No
need
having
charge
not
night
lowest
small
Needle
swimming
quiet
Comfy
pretty
small
service
night
no
single
cooked
shade
quiet
I
needed
next
Quarter
block
out
always
an
job
little
missed
top
huge
d
stay
fancier
top
swimming
By
food
an
view
not
complementary
insipid
swimming
quiet
thing
Always
service
average
afterthought
by
break
too
handicap
by
fresh
no
scant
always
swimming
missing
longer
other
need
small
visit
shopping
same
stayed
smelled
dry
q

not
slowness
shopping
left
pulling
service
assuming
having
fresh
out
miss
no
missed
overlooked
neither
smell
lot
quiet
had
size
left
No
disruption
expansive
awful
automatic
diverse
no
small
Only
small
no
removing
pretty
always
out
regular
regular
undrinkable
only
jam
quiet
late
cold
having
not
average
needed
only
bring
slow
James
quiet
bother
always
Quarter
night
out
terminal
cruise
cruise
complaint
hidden
not
exit
keep
not
fairly
lobby
small
another
small
service
size
view
public
night
need
save
not
welcoming
service
top
had
generous
complementary
size
open
small
little
had
had
thing
swimming
cold
service
dryer
average
an
dollar
had
by
small
swimming
an
by
block
save
comfy
had
had
small
too
service
missed
charge
ran
service
always
kid
sell
nearby
ran
kept
kept
missed
charge
single
kept
quiet
kid
regular
stay
charge
Has
lobby
expected
quiet
small
no
Fresh
swimming
double
by
by
row
Plenty
Limited
swimming
head
Small
no
no
swimming
no
always
No
option
No
option
blow
dryer
experience
swim

## 3. More

In [42]:
#!pip install negspacy
import spacy
from negspacy.negation import Negex

nlp = spacy.load("en_core_web_sm")
negex = Negex(nlp, ent_types=["NOUN','PRODUCT"])
nlp.add_pipe(negex, last=True)

doc = nlp("Like Carmel, no air-conditioner in rooms but they have a fan for air circulation. Nice pool, nice food, bad car service.")
for e in doc.ents:
    print(e.text, e._.negex)

Carmel False


In [43]:
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(reviews):
    stop_free = " ".join([i for i in reviews.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

reviews_clean = [clean(reviews).split() for reviews in reviews]    

dictionary = corpora.Dictionary(reviews_clean)

In [44]:
reviews_clean

[['hotel',
  'nice',
  'quiet',
  'know',
  'train',
  'track',
  'near',
  'by',
  'train',
  'passed',
  'stay',
  'best',
  'western',
  'changed',
  'hotel',
  'classification',
  'plus',
  'category',
  'before'],
 ['stayed',
  'king',
  'suite',
  'separation',
  'bedroom',
  'living',
  'space',
  'sofa',
  'bed',
  'good',
  'back',
  'discomfort',
  'day',
  'left',
  'three',
  'night',
  'stay',
  'room',
  'clean',
  'king',
  'bed',
  'comfortable',
  'hotel',
  'located',
  'within',
  'walking',
  'distance',
  'place',
  'want',
  'to'],
 ['parking',
  'horrible',
  'somebody',
  'ran',
  'rental',
  'car',
  'staying',
  'there',
  'get',
  'try',
  'breakfast',
  'business',
  'restaurant',
  'opened',
  'late',
  'business',
  'world',
  'enjoy',
  'asked',
  'coffee',
  'room',
  'item',
  'vending',
  'machine',
  'stale'],
 ['cheap',
  'excellent',
  'location',
  'price',
  'somewhat',
  'standard',
  'hacing',
  'reservation',
  'room',
  'nice',
  'clean',
  'o

In [45]:
dictionary

In [46]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

reviews_term_matrix = [dictionary.doc2bow(reviews) for reviews in reviews_clean]

In [47]:
print(reviews_term_matrix)
for i in reviews_term_matrix:
    print(i)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1)], [(6, 1), (13, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(32, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)], [(9, 1), (11, 1), (20, 1), (25, 1), (32, 2), (44, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(8, 1), (13, 3), (22, 2), (25, 1), (32, 3), (41, 1), (49, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95,

In [60]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(reviews_term_matrix, num_topics=8, id2word = dictionary, passes=5)

In [61]:
print(ldamodel.print_topics(num_topics=8, num_words=5))

[(0, '0.051*"room" + 0.051*"excellent" + 0.027*"good" + 0.027*"clean" + 0.027*"price"'), (1, '0.055*"room" + 0.055*"stay" + 0.042*"day" + 0.029*"good" + 0.029*"get"'), (2, '0.012*"business" + 0.012*"enjoy" + 0.012*"rental" + 0.012*"train" + 0.011*"room"'), (3, '0.012*"room" + 0.012*"excellent" + 0.011*"reservation" + 0.011*"front" + 0.011*"clean"'), (4, '0.010*"room" + 0.010*"hotel" + 0.010*"stay" + 0.010*"get" + 0.010*"good"'), (5, '0.011*"bed" + 0.011*"king" + 0.010*"day" + 0.010*"to" + 0.010*"left"'), (6, '0.064*"hotel" + 0.064*"train" + 0.038*"stay" + 0.035*"near" + 0.034*"best"'), (7, '0.056*"business" + 0.030*"stale" + 0.030*"item" + 0.030*"vending" + 0.029*"breakfast"')]
